In [ ]:
import math
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import style
import matplotlib
import time
import scanpy as sc
import sklearn
import networkx as nx
import ot
import paste as pst
#%load_ext autoreload
#%autoreload 2
# style.use('seaborn-dark')
style.use('seaborn-white')

In [ ]:
def largest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return np.unravel_index(indices, ary.shape)

def plot2D_samples_mat(xs, xt, G, thr=1e-8,alpha=0.2,top=1000,weight_alpha=False,**kwargs):
    if ('color' not in kwargs) and ('c' not in kwargs):
        kwargs['color'] = 'k'
    mx = G.max()
    idx = largest_indices(G,top)
    for l in range(len(idx[0])):
        plt.plot([xs[idx[0][l], 0], xt[idx[1][l], 0]], [xs[idx[0][l], 1], xt[idx[1][l], 1]],alpha=alpha*(1-weight_alpha)+(weight_alpha*G[idx[0][l],idx[1][l]] /mx),c='k')

    
def plot_slice_pairwise_alignment(slice1,slice2,pi,thr=1-1e-8,alpha=0.05,top=1000,name='',save=False,weight_alpha=False):
    coordinates1,coordinates2 = slice1.obsm['spatial'],slice2.obsm['spatial']
    offset = (coordinates1[:,0].max()-coordinates2[:,0].min())*1.1
    temp = np.zeros(coordinates2.shape)
    temp[:,0] = offset
    plt.figure(figsize=(20,10))
    plot2D_samples_mat(coordinates1, coordinates2+temp, pi,thr=thr, c='k',alpha=alpha,top=top,weight_alpha=weight_alpha)
    plt.scatter(coordinates1[:,0],coordinates1[:,1],linewidth=0,s=100, marker=".",color=list(slice1.obs['layer_guess_reordered'].map(dict(zip(slice1.obs['layer_guess_reordered'].cat.categories,slice1.uns['layer_guess_reordered_colors'])))))
    plt.scatter(coordinates2[:,0]+offset,coordinates2[:,1],linewidth=0,s=100, marker=".",color=list(slice2.obs['layer_guess_reordered'].map(dict(zip(slice2.obs['layer_guess_reordered'].cat.categories,slice2.uns['layer_guess_reordered_colors'])))))
    plt.gca().invert_yaxis()
    plt.axis('off')
    plt.show()

## Organize data

### Run PASTE `pairwise_align`.

## Rotate and plot new coordinates

In [ ]:
paste_layer_groups = [pst.stack_slices_pairwise(layer_groups[j], pis[j]) for j in range(len(layer_groups)) ]

In [ ]:
def plot_slices_overlap(groups, adatas, sample_list, layer_to_color_map):
    for j in range(len(groups)): 
        plt.figure(figsize=(10,10))
        for i in range(len(groups[j])):
            adata = adatas[sample_list[j*4+i]]
            colors = list(adata.obs['layer_guess_reordered'].astype('str').map(layer_to_color_map))
            plt.scatter(groups[j][i].obsm['spatial'][:,0],groups[j][i].obsm['spatial'][:,1],linewidth=0,s=100, marker=".",color=colors)
        plt.legend(handles=[mpatches.Patch(color=layer_to_color_map[adata.obs['layer_guess_reordered'].cat.categories[i]], label=adata.obs['layer_guess_reordered'].cat.categories[i]) for i in range(len(adata.obs['layer_guess_reordered'].cat.categories))],fontsize=10,title='Cortex layer',title_fontsize=15,bbox_to_anchor=(1, 1))
        plt.gca().invert_yaxis()
        plt.axis('off') 
        plt.show()

Below we plot stacking all slices of each DLPFC sample wihout alignment and with PASTE alignment. We also showcase how the STUtility alignment compares later in the notebook when we benchmark against other methods. 

In [ ]:
# Plot Stacking of Four slices without alignment
plot_slices_overlap(layer_groups, adatas, sample_list, layer_to_color_map)

In [ ]:
# Plot Stacking of Four slices with PASTE alignment
plot_slices_overlap(paste_layer_groups, adatas, sample_list, layer_to_color_map)

In [ ]:
import plotly.express as px
import plotly.io as pio
# if using plain jupyter notebook use:
pio.renderers.default='notebook'
# If using jupyterlab use:
#pio.renderers.default='jupyterlab'
config = {
      'toImageButtonOptions': {
        'format': 'svg', # one of png, svg, jpeg, webp
        'filename': "DLPFC_3d"
      }
    }

factor = 6.1656/100
z_difference = [np.array([0,10,300,10])*factor for j in range(len(paste_layer_groups))]
z_coordinates = [[z_difference[j][:i+1].sum() for i in range(len(z_difference[j]))] for j in range(len(z_difference))]
z_coordinates = np.array([[0,1,2,3] for j in range(3)])*75

for j in range(len(paste_layer_groups)): 
    df = pd.DataFrame(columns=['x','y','z','layer'])
    for i,L in enumerate(paste_layer_groups[j]):
        adata = adatas[sample_list[j*4+i]]
        df_ = pd.DataFrame(np.concatenate((L.obsm['spatial'],np.matrix(z_coordinates[j][i]*np.ones(len(adata))).T,np.matrix(np.ones(len(adata),dtype=int)).T),axis=1),columns=['x','y','z','layer'])
        df_.loc[:,"layer"] = list(adata.obs['layer_guess_reordered'])
        df = df.append(df_, ignore_index=True)
    df = df.sort_values(by=['layer'])
    df['y'] = -df['y']
    df[['x','y','z']] = df[['x','y','z']]/factor
    fig = px.scatter_3d(df, x='x', y='y', z='z',
                  color='layer',color_discrete_sequence=[matplotlib.colors.to_hex(layer_to_color_map[c]) for c in sorted(list(adata.obs['layer_guess_reordered'].cat.categories))])#adata.uns['layer_guess_reordered_colors'])
    fig.update_layout(scene_aspectmode='data',font=dict(size=8))
    fig.show(config=config)